# Face Detector (work in progress)
### Testing face detector using "face_angles.jpg"

In [22]:
# getting image path
# try with flowers.jpg as well (doesn't have any faces)
current = %pwd
from pathlib import Path
path = Path(current)
imagePath = str(path.parent.absolute())
image_name = "face_angles.jpg"
imagePath += '\\' + image_name
imagePath

'c:\\Users\\brian\\Desktop\\PYUM\\P-YUM\\face_angles.jpg'

In [23]:
cascPath = "haarcascade_frontalface_default.xml"
cascPath_m = "haarcascade_mouth.xml"
cascPath_e = "haarcascade_eye.xml"

In [24]:
import cv2

In [37]:
# Create the haar cascade
faceCascade = cv2.CascadeClassifier(cascPath)
mouthCascade = cv2.CascadeClassifier(cascPath_m)
eyeCascade = cv2.CascadeClassifier(cascPath_e)

# Read the image
image = cv2.imread(imagePath)
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Detect faces in the image
faces = faceCascade.detectMultiScale(
    gray,
    scaleFactor=1.1,
    minNeighbors=5,
    minSize=(30, 30),
    flags = cv2.CASCADE_SCALE_IMAGE
)

mouth = mouthCascade.detectMultiScale(
    gray,
    scaleFactor=1.1,
    minNeighbors=5,
    minSize=(30, 30),
    flags = cv2.CASCADE_SCALE_IMAGE
)

eyes = eyeCascade.detectMultiScale(
    gray,
    scaleFactor=1.1,
    minNeighbors=5,
    minSize=(30, 30),
    flags = cv2.CASCADE_SCALE_IMAGE
)

In [38]:
print ("Found {0} faces!".format(len(faces)))

# Draw a rectangle around the faces
for (x, y, w, h) in faces:
    cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)

print ("Found {0} mouths!".format(len(mouths)))

# Draw a rectangle around the faces
for (x, y, w, h) in mouths:
    cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)

print ("Found {0} eyes!".format(len(eyes)))

# Draw a rectangle around the faces
for (x, y, w, h) in eyes:
    cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)

Found 9 faces!
Found 29 mouths!
Found 7 eyes!


In [39]:
# runs reasonably well
# notice how it doesn't really work on faces that are turned 90 degrees.

cv2.imshow("Features found", image)
cv2.waitKey(0)


-1

# Face As A Percentage of the Video

In [15]:
# first, get the path of the video in question
# getting video path
current = %pwd
from pathlib import Path
path = Path(current)
filename = str(path.parent.absolute())
video_name = "American Grilled Cheese Vs. French Grilled Cheese.mp4"
filename += '\\' + video_name
filename

'c:\\Users\\brian\\Desktop\\PYUM\\P-YUM\\American Grilled Cheese Vs. French Grilled Cheese.mp4'

In [16]:
import cv2
import numpy as np

In [17]:
def face_detector(image):
    cascPath = "haarcascade_frontalface_default.xml"

    # Create the haar cascade
    faceCascade = cv2.CascadeClassifier(cascPath)

    # Read the image
    # image = cv2.imread(image)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Detect faces in the image
    faces = faceCascade.detectMultiScale(
        gray,
        scaleFactor=1.3,
        minNeighbors=5,
        minSize=(30, 30),
        flags = cv2.CASCADE_SCALE_IMAGE
    )

    for (x, y, w, h) in faces:
        cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 2)

    if len(faces) >= 1:
        cv2.imshow("Faces found", image)
        cv2.waitKey(0)


    # if it detects at least one face, return true
    # it doesn't have to be the host itself
    if len(faces) >= 1:
        return True
    
    return False

In [18]:
# input: the video file
# output: length of video file in seconds

def find_video_length(video):

    fps = video.get(cv2.CAP_PROP_FPS)
    frame_count = video.get(cv2.CAP_PROP_FRAME_COUNT)

    return frame_count / fps, fps


In [19]:
def face_percentage(filename):

    # person_counter counts number of frames person was in shot
    person_counter = 0
    # no_person_counter counts number of frames where person is not in shot
    no_person_counter = 0

    my_video = cv2.VideoCapture(filename)
    if (my_video.isOpened() == False):
        print("Error opening video stream or file")

    video_length, video_fps = find_video_length(my_video)

    nth_frame = video_fps // 2
    frame_counter = 0

    while (my_video.isOpened()):
        
        ret, frame = my_video.read()
        if frame_counter % nth_frame == 0:

            # Capture frame-by-frame
            if ret == True:
                if face_detector(frame):

                    print(frame_counter / video_fps)
                    person_counter += 1
                else:
                    no_person_counter += 1
            else:
                break

        if ret == False:
            break

        frame_counter += 1

    return (person_counter * 100) / (no_person_counter + person_counter)




In [20]:
filename

'c:\\Users\\brian\\Desktop\\PYUM\\P-YUM\\American Grilled Cheese Vs. French Grilled Cheese.mp4'

In [21]:
face_percentage(filename)

0.46713333333333334
0.9342666666666667
1.4014
1.8685333333333334
2.8028
3.2699333333333334
3.7370666666666668
4.2042
4.671333333333333
5.138466666666667
18.685333333333332
19.152466666666665
19.619600000000002
23.823800000000002
24.290933333333335
24.758066666666668
25.2252
25.692333333333334
27.093733333333333
27.560866666666666
28.495133333333335
28.962266666666668
29.4294
29.896533333333334
30.8308
31.297933333333333
31.765066666666666
35.035000000000004
35.50213333333333
35.96926666666667
36.4364
36.903533333333336
37.370666666666665
37.8378
39.239200000000004
39.70633333333333
40.17346666666667
41.107733333333336
41.574866666666665
43.443400000000004
43.91053333333333
44.37766666666667


KeyboardInterrupt: 